# Network component analysis

In [13]:
import pandas as pd
import numpy as np
from pathlib import Path
import networkx as nx
from networkx.algorithms import community
from tqdm import tqdm
from itertools import tee

Path.mkdir(Path("analysis"), exist_ok=True)

In [9]:
edgelists = []

for file in Path("edgelistsSample").iterdir():
    tmp_edgelist = pd.read_csv(file)
    edgelists.append(tmp_edgelist)

In [12]:
nx_graphs = []

for edgelist in edgelists:
    tmp_nx_graph = nx.from_pandas_edgelist(edgelist, source='Source', target='Target', create_using=nx.Graph())
    nx.set_edge_attributes(tmp_nx_graph, 1, 'weight')
    nx_graphs.append(tmp_nx_graph)

In [24]:
# Read csv from edgelistsSample folder

edgelists_sample_df = pd.read_csv("edgelistsSample/edgelists_any_sample.csv")
edgelists_sample_df
G = nx.from_pandas_edgelist(edgelists_sample_df, source='Source', target='Target', create_using=nx.Graph())
# Set all weights to 1
nx.set_edge_attributes(G, 1, 'weight')
print(len(G.nodes))

359703


In [25]:
louvain_nx = community.louvain_partitions(G)
first = next(louvain_nx)
print(f"After first passage: {len(first)} communities")
second = next(louvain_nx)
print(f"After second passage: {len(second)} communities")

After first passage: 43 communities
After second passage: 33 communities


In [47]:
# Louvain implementation
def louvain(G, npassage):
    # Will contain the graph and the communities after each passage
    data = {}
    for i in range(npassage):
        print(f"Passage {i}", flush=True)
        old_G, G, communities, get_community = louvain_step(G, G, i==0)
        data[i] = (old_G, communities, get_community)
        print(f"There are {len(communities)} communities after passage {i}", flush=True)
    return data


def louvain_step(G, copy, is_first_passage):
    # Step 1: Initialization, start with each node being a single community
    communities = {idx: set([node]) for idx, node in enumerate(G.nodes)}
    # To get direct access to the community (it speeds up a bit the algorithm)
    get_community = {node: idx for idx, node in enumerate(G.nodes)}
    # Used in the modularity computation
    neighbors_sets = {node: set(G.neighbors(node)) for node in G.nodes}
    m = len(G.edges)

    # Sum the weights of the incident edges for all nodes inside a community, for all communities
    # Separate first passage and other ones to speed up the algorithm
    if is_first_passage:
        sum_communities = {idx: sum(dict(G.degree(community)).values()) for idx, community in communities.items()}
    else:
        sum_communities = {idx: sum(dict(G.degree(community, 'weight')).values()) for idx, community in communities.items()}

    # Fixed number of iterations
    # TODO: change it?
    for i in range(4):
        print(f"Iteration {i}", flush=True)
        for node in tqdm(G.nodes):
            # Step 2: Remove node from its community
            # TODO: make sure what we want to do if the node has no neighboring communities
            neighboring_communities = get_neighboring_communities(G, node, get_community)
            if neighboring_communities == set():
                continue
            belong_to = get_community[node]
            communities[belong_to].remove(node)
            sum_communities[belong_to] -= G.degree(node, 'weight')
            if communities[belong_to] == set():
                del communities[belong_to]
                del sum_communities[belong_to]

            # Step 3: Insert the node in the community that maximizes the modularity
            scores = [
                (neighbor_community, modularity_gain(G, node, communities[neighbor_community], sum_communities[neighbor_community], neighbors_sets[node], m, is_first_passage))
                for neighbor_community in neighboring_communities
            ]
            best_community, best_score = max(scores, key=lambda x: x[1])
            communities[best_community].add(node)
            get_community[node] = best_community
            sum_communities[best_community] += G.degree(node, 'weight')

    return G, get_new_graph(G, communities, sum_communities, get_community), communities, get_community


def get_neighboring_communities(G, node, get_community):
    # Use a set to make sure a community only appear once
    neighboring_communities = set()
    for neighbor in G.neighbors(node):
        if neighbor == node: continue
        neighboring_communities.add(get_community[neighbor])

    return neighboring_communities


def modularity_gain(G, node, community, sum_community, neighbor_set, m, is_first_passage):
    # Separate first passage and other ones to speed up the algorithm
    if is_first_passage:
        # Sum the weights of the edges from node into community nodes
        # Using sets allow to use intersection()
        sum_weights_node = len(neighbor_set.intersection(community))
        right_member = (sum_community * G.degree[node]) / (2 * (m**2))
    else:
        # Sum the weights of the edges from node into community nodes
        sum_weights_node = sum([G.get_edge_data(node, member)['weight'] for member in G.neighbors(node) if member in community])
        right_member = (sum_community * G.degree(node, 'weight')) / (2 * (m**2))
    # Compute modularity
    left_member = sum_weights_node / (2 * m)
    return left_member - right_member


def get_new_graph(old_G, communities, sum_communities, get_community):
    print("Construct new graph", flush=True)
    G = nx.Graph()
    G.add_nodes_from(communities.keys())
    for community in communities:
        G.add_edge(community, community, weight=sum_communities[community])

    for source, dest, weight_dict in old_G.edges(data=True):
        community1 = get_community[source]
        community2 = get_community[dest]
        current_weight = G.get_edge_data(community1, community2, {'weight': 0})['weight']
        # TODO: is this correct?
        new_weight = current_weight + weight_dict['weight']
        G.add_edge(community1, community2, weight=new_weight)
    return G

In [48]:
results = []
for idx, G in enumerate(nx_graphs):
    print(f"Louvain on {idx+1}/{len(nx_graphs)}", flush=True)
    data = louvain(G, 1)
    results.append(data)

Louvain on 1/3
Passage 0
Iteration 0


100%|██████████| 718105/718105 [00:23<00:00, 30399.89it/s]

Iteration 1



100%|██████████| 718105/718105 [00:22<00:00, 31263.01it/s]

Iteration 2


...


100%|██████████| 90487/90487 [00:03<00:00, 29844.94it/s]

Iteration 3



100%|██████████| 90487/90487 [00:03<00:00, 29306.25it/s]

Construct new graph


There are 100 communities after passage 0


In [52]:
test = results[0][0]
networkx_graph, communities_dict, get_community = test

# convert get_community to pandas dataframe

communities_df

,Id,Community,Label,Is_author
0,1467218983405465603,1,1467218983405465603,False
1,1369587782897831939,1,1369587782897831939,False
2,1324887871484997633,1,1324887871484997633,False
3,1641262814,1,1641262814,False
4,1298969733174366209,1,1298969733174366209,False
...,...,...,...,...
718100,2534082858,680777,2534082858,False
718101,15918451,680777,15918451,False
718102,348797527,680777,348797527,False
718103,2886355146,680777,2886355146,False


In [54]:
for idx, result in enumerate(results):
    first_entry = result[0]
    networkx_graph, communities_dict, get_community = first_entry

    communities_df = pd.DataFrame(get_community.items(), columns=['Id', 'Community'])
    communities_df["Label"] = communities_df["Id"].map(lambda x: x)
    communities_df["Is_author"] = False

    # communities_df = pd.DataFrame(columns=["Id","Label", "Community", "Is_author"])

    # communities_df = pd.DataFrame(get_community.items(), columns=['Id', 'community'])

    # for key, value in communities_dict.items():
    #     for idx in value:
    #         communities_df = pd.concat([communities_df, pd.DataFrame({"Id":idx, "Label":idx, "Community":key, "Is_author":False}, index=[0])], ignore_index=True)
    
    communities_df.loc[communities_df['Id'].isin(edgelists[idx]['Target'].unique()), "Is_author"] = True

    communities_df.to_csv(f"analysis/communities_df_{idx}.csv", index=False)

## TODO:
- Sample by taking random author_id and take all edges connected to it
- Implement from scratch community detection: Louvain (faster than Girvan)
- Look at how to store the communities (edgelist or so)
- Look at the caracteristics of the huge communities: what is the Tweet, who is the author, what are the hashtags etc...
- Make profiles of users: press, random guy, famous guy, etc...
- Look at outliers
- Load communities to Gephi to vizualise them